Este processo irá realizar a limpeza a agregação dos dados de interação dos Clientes B2B com a plataforma

In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

Limpeza de dados de Clientes do site

In [3]:
df_account = pd.read_csv('Data/Account.csv')

Limpeza de Produtos

In [4]:
df_product = pd.read_csv('Data/Product.csv')

/tmp/ipykernel_30429/4285649015.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_product = pd.read_csv('Data/Product.csv')


Limpeza e Agregação de dados de Pedidos

In [5]:
df_order = pd.read_csv('Data/Order.csv')
df_order['Type'] = 'Order'
df_order['Interaction'] = 2
df_order_item = pd.read_csv('Data/OrderItem.csv')
df_order_interaction = pd.merge(df_order, df_order_item, how='inner', on='OrderId')
df_order_product_interaction = pd.merge(df_order_interaction, df_product, how='inner', on='ProductId')

Limpeza e Agregação de dados de Carrinho de Compras

In [6]:
df_cart = pd.read_csv('Data/WebCart.csv')
df_cart['Type'] = 'Cart'
df_cart['Interaction'] = 1
df_cart_item = pd.read_csv('Data/CartItem.csv')
df_cart_interaction = pd.merge(df_cart, df_cart_item, how='inner', on='WebCartId')
df_cart_product_interaction = pd.merge(df_cart_interaction, df_product, how='inner', on='ProductId')

In [7]:
df_interactions = pd.concat([df_cart_product_interaction, df_order_product_interaction], ignore_index=True)

In [8]:
# Aqui devemos utilizar o inner para remover as contas que não possuem nenhuma interação
df_account_interactions = pd.merge(df_account, df_interactions, how='inner', left_on='Id', right_on='AccountId') 

In [9]:
df_cleaned_interactions = df_account_interactions.drop(['Id', 'CartItemId', 'ProductId', 'Discontinued', 'OrderItemId'], axis=1)

In [10]:
df_cleaned_interactions['OrderDate'] = pd.to_datetime(df_cleaned_interactions['OrderDate'], errors='coerce') 
df_cleaned_interactions['CartDate'] = pd.to_datetime(df_cleaned_interactions['CartDate'].astype(str).str[:10], errors='coerce') 

In [11]:
df_cleaned_interactions['OrderRecency'] = (datetime.now() - df_cleaned_interactions[df_cleaned_interactions['OrderDate'] < '2025-09-01']['OrderDate']).dt.days
# Nesta etapa transforma a recência menor que 1 dia em um valor próximo a zero, para que o cálculo de recência não retorne nulo (Divisão por Zero)
df_cleaned_interactions['OrderRecency'] = 1 / df_cleaned_interactions['OrderRecency'].replace(0,0.0000001)

In [12]:
df_cleaned_interactions['CartRecency'] = (datetime.now() - df_cleaned_interactions['CartDate']).dt.days
# Nesta etapa transforma a recência menor que 1 dia em um valor próximo a zero, para que o cálculo de recência não retorne nulo (Divisão por Zero)
df_cleaned_interactions['CartRecency'] = 1 / df_cleaned_interactions['CartRecency'].replace(0,0.0000001)

In [13]:
# Cria coluna que indica que interações ocorreram no mês de Setembro (Em produção seria alterado para o mês corrente)
current_month_cart = (
    (df_cleaned_interactions['Type'] == 'Cart') &
    (df_cleaned_interactions['CartDate'].dt.year == 2025) &
    (df_cleaned_interactions['CartDate'].dt.month == 9)
)

current_month_order = (
    (df_cleaned_interactions['Type'] == 'Order') &
    (df_cleaned_interactions['OrderDate'].dt.year == 2025) &
    (df_cleaned_interactions['OrderDate'].dt.month == 9)
)

df_cleaned_interactions['InteractionCurrentMonth'] = np.select(
    [current_month_cart, current_month_order],
    [1, 2],
    default=0
)

In [14]:
current_date = datetime.now()

# Correctly calculate date ranges for 3 and 6 months
three_months_ago = current_date - pd.DateOffset(months=3)
six_months_ago = current_date - pd.DateOffset(months=6)

is_3_month = (df_cleaned_interactions['OrderDate'] >= three_months_ago) &  ~current_month_order
is_6_month = (df_cleaned_interactions['OrderDate'] >= six_months_ago) &  ~current_month_order

# Apply masks to create time-based total columns
df_cleaned_interactions['TotalPrice3Month'] = np.where(is_3_month, df_cleaned_interactions['NetValue'], 0)
df_cleaned_interactions['TotalPrice6Month'] = np.where(is_6_month, df_cleaned_interactions['NetValue'], 0)

df_cleaned_interactions['TotalQuantity3Month'] = np.where(is_3_month, df_cleaned_interactions['Quantity'], 0)
df_cleaned_interactions['TotalQuantity6Month'] = np.where(is_6_month, df_cleaned_interactions['Quantity'], 0)

df_cleaned_interactions['CountOrder3Month'] = np.where(is_3_month, 1, 0)
df_cleaned_interactions['CountOrder6Month'] = np.where(is_6_month, 1, 0)

df_cleaned_interactions['OrderNetValue'] = np.where(df_cleaned_interactions['Type'] == 'Order', df_cleaned_interactions['NetValue'], 0)
df_cleaned_interactions['OrderQuantity'] = np.where(df_cleaned_interactions['Type'] == 'Order', df_cleaned_interactions['Quantity'], 0)

In [15]:
is_3_month = (df_cleaned_interactions['CartDate'] >= three_months_ago) &  ~current_month_cart
is_6_month = (df_cleaned_interactions['CartDate'] >= six_months_ago) &  ~current_month_cart

# Apply masks to create time-based total columns
df_cleaned_interactions['TotalPriceCart3Month'] = np.where(is_3_month, df_cleaned_interactions['NetValue'], 0)
df_cleaned_interactions['TotalPriceCart6Month'] = np.where(is_6_month, df_cleaned_interactions['NetValue'], 0)

df_cleaned_interactions['TotalQuantityCart3Month'] = np.where(is_3_month, df_cleaned_interactions['Quantity'], 0)
df_cleaned_interactions['TotalQuantityCart6Month'] = np.where(is_6_month, df_cleaned_interactions['Quantity'], 0)

df_cleaned_interactions['CountCart3Month'] = np.where(is_3_month, 1, 0)
df_cleaned_interactions['CountCart6Month'] = np.where(is_6_month, 1, 0)

df_cleaned_interactions['CartNetValue'] = np.where(df_cleaned_interactions['Type'] == 'Cart', df_cleaned_interactions['NetValue'], 0)
df_cleaned_interactions['CartQuantity'] = np.where(df_cleaned_interactions['Type'] == 'Cart', df_cleaned_interactions['Quantity'], 0)

In [16]:
df_grouped_interactions = (
    df_cleaned_interactions
        .groupby(['AccountId', 'ProductCode'])
        .agg(
            NetValue=('NetValue', 'sum'),
            Quantity=('Quantity', 'sum'),
            Recency=('OrderRecency', 'min'),
            CartRecency=('CartRecency', 'min'),
            Weight=('Weight', 'first'),
            ProductLine=('ProductLine', 'first'),
            Size=('Size', 'first'),
            TotalPrice3Month=('TotalPrice3Month', 'sum'),
            TotalPrice6Month=('TotalPrice6Month', 'sum'),
            TotalQuantity3Month=('TotalQuantity3Month', 'sum'),
            TotalQuantity6Month=('TotalQuantity6Month', 'sum'),
            CountOrder3Month=('CountOrder3Month', 'sum'),
            CountOrder6Month=('CountOrder6Month', 'sum'),
            OrderClassification=('InteractionCurrentMonth', 'max'),
            OrderQuantity=('OrderQuantity', 'sum'),
            CartQuantity=('CartQuantity', 'sum'),
            TotalPriceCart3Month=('TotalPriceCart3Month', 'sum'),
            TotalPriceCart6Month=('TotalPriceCart6Month', 'sum'),
            TotalQuantityCart3Month=('TotalQuantityCart3Month', 'sum'),
            TotalQuantityCart6Month=('TotalQuantityCart6Month', 'sum'),
            CountCart3Month=('CountCart3Month', 'sum'),
            CountCart6Month=('CountCart6Month', 'sum'),
            CartNetValue=('CartNetValue', 'sum'),
            OrderNetValue=('OrderNetValue', 'sum'),
            CountOrder=('OrderId', 'nunique'),
            CountCart=('WebCartId', 'nunique'),
        )
).reset_index()

In [17]:
# --- 5. Post-Aggregation Calculations ---
# Calculate Average Tickets, handling potential division by zero
df_grouped_interactions['AverageOrderTicket'] = df_grouped_interactions['OrderNetValue'] / df_grouped_interactions['CountOrder'].replace(0, np.nan)
df_grouped_interactions['AverageOrderTicket3Month'] = df_grouped_interactions['TotalPrice3Month'] / df_grouped_interactions['CountOrder3Month'].replace(0, np.nan)
df_grouped_interactions['AverageOrderTicket6Month'] = df_grouped_interactions['TotalPrice6Month'] / df_grouped_interactions['CountOrder6Month'].replace(0, np.nan)

In [18]:
df_grouped_interactions['AverageCartTicket'] = df_grouped_interactions['CartNetValue'] / df_grouped_interactions['CountCart'].replace(0, np.nan)
df_grouped_interactions['AverageCartTicket3Month'] = df_grouped_interactions['TotalPriceCart3Month'] / df_grouped_interactions['CountCart3Month'].replace(0, np.nan)
df_grouped_interactions['AverageCartTicket6Month'] = df_grouped_interactions['TotalPriceCart6Month'] / df_grouped_interactions['CountCart6Month'].replace(0, np.nan)

In [19]:

# Calculate Product Popularity (Dense Rank by NetValue within each product group)
df_grouped_interactions['ProductPopularity'] = df_grouped_interactions.groupby('ProductCode')['NetValue'].rank(method='dense', ascending=False).astype(int)

In [20]:
# Calculate Total Price and Quantity by Account
df_totals_by_account = df_grouped_interactions.groupby('AccountId').agg(
    TotalPriceByAccount=('OrderNetValue', 'sum'),
    TotalQuantityByAccount=('OrderQuantity', 'sum')
).reset_index()
df_grouped_interactions = pd.merge(df_grouped_interactions, df_totals_by_account, on='AccountId', how='left')

In [21]:
# Calculate Relative Price metrics
total_netvalue_per_product = df_grouped_interactions.groupby('ProductCode')['OrderNetValue'].transform('sum')
df_grouped_interactions['RelativePriceProduct'] = df_grouped_interactions['OrderNetValue'] / total_netvalue_per_product.replace(0, np.nan)

total_netvalue_per_account = df_grouped_interactions.groupby('AccountId')['OrderNetValue'].transform('sum')
df_grouped_interactions['RelativePriceAccount'] = df_grouped_interactions['OrderNetValue'] / total_netvalue_per_account.replace(0, np.nan)

In [22]:
# Round float columns to 2 decimal places
float_cols = df_grouped_interactions.select_dtypes(include=['float64']).columns
df_grouped_interactions[float_cols] = df_grouped_interactions[float_cols].round(2)

float_cols = df_grouped_interactions.select_dtypes(include=['object']).columns
df_grouped_interactions[float_cols] = df_grouped_interactions[float_cols].fillna('')

In [23]:
df_grouped_interactions = df_grouped_interactions.sort_values(['AccountId', 'ProductCode', 'NetValue'], ascending=False)

In [24]:
df_grouped_interactions.groupby('AccountId').agg(Count=('ProductCode', 'nunique')).sort_values('Count', ascending=False)

,Count
AccountId,
0018c00002uQ4MFAA0,468
0018c00002nZUgVAAW,466
001U400000J7k2JIAR,414
001U400000Beo31IAB,392
001U400000TnJwtIAF,390
...,...
001U400000LPjltIAD,1
001U4000008SRC0IAO,1
001U4000008Sx2JIAS,1


In [26]:
dfs = []

all_products = df_product.groupby('ProductCode').agg(ProductLine=('ProductLine', 'first'), Size=('Size', 'first'))

for acc_id, group in df_grouped_interactions.groupby('AccountId'):
    n_current = len(group)
    n_to_add = max(1, int(n_current * 0.10))  # 10%
    top_product_line = max(group['ProductLine'])

    df_product_by_line = (all_products[all_products['ProductLine'] == top_product_line]).reset_index()
    top_product_size = max(df_product_by_line['Size'].fillna(''))

    random_products = np.random.choice(df_product_by_line['ProductCode'], size=n_to_add, replace=True)
    
    new_rows = pd.DataFrame({
        'AccountId': [acc_id] * n_to_add,
        'ProductCode': random_products,
        'ProductLine': [top_product_line] * n_to_add,
        'Size': [top_product_size] * n_to_add
    })
    
    group = pd.concat([group, new_rows], ignore_index=True)
    dfs.append(group)

df_final = pd.concat(dfs, ignore_index=True)


In [27]:
# Round float columns to 2 decimal places
float_cols = df_final.select_dtypes(include=['float64']).columns
df_final[float_cols] = df_final[float_cols].fillna(0)

In [28]:
df_final.head(10)

,AccountId,ProductCode,NetValue,Quantity,Recency,CartRecency,Weight,ProductLine,Size,TotalPrice3Month,...,AverageOrderTicket3Month,AverageOrderTicket6Month,AverageCartTicket,AverageCartTicket3Month,AverageCartTicket6Month,ProductPopularity,TotalPriceByAccount,TotalQuantityByAccount,RelativePriceProduct,RelativePriceAccount
0,0018c00002dJJv0AAG,92314010,3383.52,2.0,0.00,0.0,83.60,Lar,,0.00,...,0.00,0.00,0.0,0.0,0.0,43.0,12198.12,8.0,0.02,0.28
1,0018c00002dJJv0AAG,92210010,4540.80,3.0,0.00,0.0,76.80,Lar,,0.00,...,0.00,0.00,0.0,0.0,0.0,24.0,12198.12,8.0,0.05,0.37
2,0018c00002dJJv0AAG,92015010,4273.80,3.0,0.00,0.0,74.80,Lar,,0.00,...,0.00,0.00,0.0,0.0,0.0,37.0,12198.12,8.0,0.04,0.35
3,0018c00002dJJv0AAG,93885100,0.00,0.0,0.00,0.0,0.00,Lar,Pequeno,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00
4,0018c00002dtWCbAAM,58026327,3253.22,7.0,0.00,0.0,9.80,Elétrica,,0.00,...,0.00,987.12,0.0,0.0,0.0,1.0,218295.68,1890.0,0.25,0.01
5,0018c00002dtWCbAAM,58026325,1550.75,5.0,0.00,0.0,5.00,Elétrica,,0.00,...,0.00,652.28,0.0,0.0,0.0,1.0,218295.68,1890.0,0.35,0.01
6,0018c00002dtWCbAAM,58026231,698.64,2.0,0.01,0.0,4.60,Elétrica,,0.00,...,0.00,698.64,0.0,0.0,0.0,1.0,218295.68,1890.0,0.19,0.00
7,0018c00002dtWCbAAM,58014078,704.78,5.0,0.01,0.0,0.36,Elétrica,,704.78,...,704.78,704.78,0.0,0.0,0.0,2.0,218295.68,1890.0,0.18,0.00
8,0018c00002dtWCbAAM,58014077,909.07,10.0,0.00,0.0,0.34,Elétrica,,454.22,...,454.22,454.22,0.0,0.0,0.0,9.0,218295.68,1890.0,0.01,0.00
9,0018c00002dtWCbAAM,58013145,1102.65,3.0,0.00,0.0,1.88,Elétrica,,0.00,...,0.00,0.00,0.0,0.0,0.0,6.0,218295.68,1890.0,0.06,0.01


In [29]:
df_final.groupby('AccountId').agg(Count=('ProductCode', 'nunique')).sort_values('Count', ascending=False)

,Count
AccountId,
0018c00002uQ4MFAA0,514
0018c00002nZUgVAAW,512
001U400000J7k2JIAR,455
001U400000Beo31IAB,431
001U400000TnJwtIAF,428
...,...
001U400000LPjltIAD,2
001U4000008SRC0IAO,2
001U4000008Sx2JIAS,2


In [30]:
df_final.to_csv('Data/AccountInteractions.csv')

In [31]:
df_final.columns

Index(['AccountId', 'ProductCode', 'NetValue', 'Quantity', 'Recency',
       'CartRecency', 'Weight', 'ProductLine', 'Size', 'TotalPrice3Month',
       'TotalPrice6Month', 'TotalQuantity3Month', 'TotalQuantity6Month',
       'CountOrder3Month', 'CountOrder6Month', 'OrderClassification',
       'OrderQuantity', 'CartQuantity', 'TotalPriceCart3Month',
       'TotalPriceCart6Month', 'TotalQuantityCart3Month',
       'TotalQuantityCart6Month', 'CountCart3Month', 'CountCart6Month',
       'CartNetValue', 'OrderNetValue', 'CountOrder', 'CountCart',
       'AverageOrderTicket', 'AverageOrderTicket3Month',
       'AverageOrderTicket6Month', 'AverageCartTicket',
       'AverageCartTicket3Month', 'AverageCartTicket6Month',
       'ProductPopularity', 'TotalPriceByAccount', 'TotalQuantityByAccount',
       'RelativePriceProduct', 'RelativePriceAccount'],
      dtype='object')